In [1]:
"""We are going to connect to the Geoserver and upload some data"""
# info source: https://pypi.org/project/geoserver-rest/

import os
import paramiko

In [2]:
"""WARNING: No hagas push de esta información"""
server_ip = "192.168.250.122"
username = ""
password = ""



In [3]:
def start_ssh_client(server_ip, username, password):
    ssh_client = paramiko.SSHClient()
    ssh_client.load_system_host_keys()
    ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh_client.connect(server_ip, username=username, password=password)
    return ssh_client

def sudo_server_petition(ssh_client, password, petition):
    # Command to execute (This is for AKIF)
    command = f'echo {password} | sudo -S {petition}'
    # Execute the command
    stdin, stdout, stderr = ssh_client.exec_command(command)
    # Capture the output
    output = stdout.read().decode().strip()
    
    return output.split('\n') # I create a list here

def server_petition(ssh_client, petition):
    # Command to execute (This is for URA)
    command = f'{petition}'
    # Execute the command
    stdin, stdout, stderr = ssh_client.exec_command(command)
    # Capture the output
    output = stdout.read().decode().strip()
    
    return output.split('\n') # I create a list here

def start_scp_client(ssh_client):
    scp_client = ssh_client.open_sftp()
    # scp_client = scp.SCPClient(ssh_client.get_transport()) # This is like an alternative
    return scp_client

def upload_file_to_server(scp_client, local_path, remote_directory):
    scp_client.put(local_path, remote_path=remote_directory)
    scp_client.close()


### Production Geoserver

In [ ]:
ssh_client = start_ssh_client(server_ip, username, password)

# Check that the geoserver_transfer is empty
read_request = r"dir /data/geoserver_transfer"
geoserver_transfer = server_petition(ssh_client, read_request)
print(geoserver_transfer)

# Pick the container name
# Esto es importante, ya que el nombre cambia ligeramente cuando se reinicia docker
read_docker_containers_request = r"docker ps --format '{{.Names}}'"
containers = server_petition(ssh_client, read_docker_containers_request)
demo_geoserver_container_list = [name for name in containers if name.startswith("demo_geoserver.")] # Cuidado que no sea el del postgis
demo_geoserver_container = str(demo_geoserver_container_list[0]) # Saca el elemento a una string
print(demo_geoserver_container)

# Optional: Get basename folders in docker container
read_docker_data_request = r"dir /data/docker-data/volumes/geoserver_data/_data/data"
geoserver_folders = server_petition(ssh_client, read_docker_data_request)
print(geoserver_folders)

In [ ]:
"""For production"""
# We asume we put everything on the same folder
local_path = r"C:\Users\admin\Downloads\dataset-satellite-land-cover\test"
remote_directory = r"\data\geoserver_transfer" 
docker_base_folder = "/opt/geoserver/data_dir/data/"
namespace_folder = "im-data-global-landcover"

for folder in os.listdir(local_path):
    # Make the folder inside the data transfer
    geoserver_transfer_folder = os.path.join(remote_directory, folder).replace(os.sep, '/')
    create_folder_request = f"mkdir {geoserver_transfer_folder}"
    server_petition(ssh_client, create_folder_request)

    # Get the files
    folder_path = os.path.join(local_path, folder)
    file_list = [file for file in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, file))]
    print("Files in the folder: ", file_list)

    # Start scp and import the resources
    scp_client = start_scp_client(ssh_client)
    for file in file_list:
        # Prepare both local and remote path (Pro TIP: Always including the file name.xx into the path)
        local_file = os.path.join(folder_path, file)
        remote_file = os.path.join(geoserver_transfer_folder, file).replace(os.sep, '/') # f"{folder_destination}/{file}"
        print(remote_file)
        scp_client.put(local_file, remote_file)

    # do the docker cp to the destination
    geoserver_transfer_folder
    docker_cp_request = f"docker cp {geoserver_transfer_folder} {demo_geoserver_container}:{docker_base_folder}{namespace_folder}"
    server_petition(ssh_client, docker_cp_request)

    # Delete the folder in the geoserver_folder
    delete_folder_request = f"rm -r {geoserver_transfer_folder}"
    server_petition(ssh_client, delete_folder_request)

scp_client.close()
ssh_client.close()

### Aux geoserver

In [9]:
"""Develop way"""
# Esto es un poco mas crissto, y como es tan inestable. Igual no es necesario desarrolarlo
ssh_client = start_ssh_client(server_ip, username, password)
# geoserver aux
# read_petittion = r"dir /data_transfer"

read_request = r"dir /samba/public/demo-geo-stack/geoserver/data/ruben"
# move_pettition = r"mv *.tif /samba/public/demo-geo-stack/geoserver/data/ruben"

result = sudo_server_petition(ssh_client, password, read_request)
print(result)

['grid.cpg  tcd_2012_020m_eu_03035_d03_full_mosaic', 'grid.dbf  tcd_2012_020m_eu_03035_d03_full_reclass.tif', 'grid.prj  tcd_2015_020m_eu_03035_d05_full_reclass_bad_projection.tif', 'grid.shp  tcd_2015_020m_eu_03035_d05_full_reclass_good_projection.tif', 'grid.shx  tcd_2018_010m_eu_03035_v2_0_mosaic']


In [11]:
read_request = r"dir /data_transfer"
read_request = r"dir /samba/public/demo-geo-stack/geoserver/data/ruben"
result = server_petition(ssh_client, read_request)
print(result)

['']


In [ ]:
"""For develop"""
# Esto está sin adaptar, es una copia
local_path = r"C:\Users\admin\Downloads\test\super_test"
remote_directory = r"/data_transfer" 

for folder in os.listdir(local_path):
    # Make the folder inside the data transfer
    geoserver_transfer_folder = os.path.join(remote_directory, folder).replace(os.sep, '/')
    create_folder_request = f"mkdir {geoserver_transfer_folder}"
    server_petition(ssh_client, create_folder_request)

    # Get the files
    folder_path = os.path.join(local_path, folder)
    file_list = [file for file in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, file))]
    print("Files in the folder: ", file_list)

    # Start scp and import the resources
    scp_client = start_scp_client(ssh_client)
    for file in file_list:
        # Prepare both local and remote path (Pro TIP: Always including the file name.xx into the path)
        local_file = os.path.join(folder_path, file)
        remote_file = os.path.join(geoserver_transfer_folder, file).replace(os.sep, '/') # f"{folder_destination}/{file}"
        print(remote_file)
        scp_client.put(local_file, remote_file)

    # move the data to the destination
    geoserver_transfer_folder
    docker_cp_request = f"docker cp {geoserver_transfer_folder} {demo_geoserver_container}:{docker_base_folder}{namespace_folder}"
    server_petition(ssh_client, docker_cp_request)

    # Delete the folder in the data_transfer
    delete_folder_request = f"rm -r {geoserver_transfer_folder}"
    server_petition(ssh_client, delete_folder_request)

scp_client.close()
ssh_client.close()